In [ ]:
%pip install pandas

In [ ]:
%pip install openpyxl

Probajmo pročitati xlsx file sa rezultatima izbora u prvoj izbornoj jedinici (csv file ima neke čudne znakove).

In [ ]:
import pandas as pd
df = pd.read_excel('data/election_results/02_01.xlsx', sheet_name='rezultati')
print(df.head())

Sada ćemo učitati rezultate svih izbornih jedinica u niz data frameova i pokupiti imena listi/stranaka. Savjet: odaberite "scrollable element" pri dnu ispisa kako biste mogli vidjeti sve stranke. Neke se ponavljaju jer imaju različite partnere u različitim izbornim jedinicama. To ćemo riješiti kasnije.

In [ ]:
dataframes = []
parties = []

file_names = [
    '02_01.xlsx', '02_02.xlsx', '02_03.xlsx', '02_04.xlsx', 
    '02_05.xlsx', '02_06.xlsx', '02_07.xlsx', 
    '02_08.xlsx', '02_09.xlsx', '02_10.xlsx'
]

for file_name in file_names:
    df = pd.read_excel(f'data/election_results/{file_name}', sheet_name='rezultati')

    #na svakom 15. mjestu, počinjući od indeksa 15, nalazi se ime liste/stranke jer trebaju imati 14 kandidata, pošto svaka izborna jedinica nosi 14 mandata
    for i in range(15, len(df.columns), 15):
      column_name = df.columns[i]
      if column_name not in parties:
          parties.append(column_name)
          
    dataframes.append(df)

#liste/stranke
for party in parties:
   print(party)

#data frame rezultata prve izborne jedinice
print(dataframes[0].head())

Sada ćemo eliminirati one stupce koji nam ne trebaju (to su primarno preferencijalni glasovi) - odnosno zadržat ćemo stupce koji nam trebaju.

In [ ]:
columns_to_keep = ['Županija', 'Grad/općina/država', 'Rbr BM', 'Naziv BM', 'Lokacija BM', 'Adresa BM', 'Ukupno birača', 'Glasovalo birača', 'Glasovalo birača (po listićima)', 'Važeći listići', 'Nevažeći listići']
columns_to_keep = columns_to_keep + parties
for i in range(len(dataframes)):
    #sve izborne jedinice nemaju iste liste/stranke - pa se inače javlja key error. Zato moramo dodati iduću liniju - radimo presjek između niza 'columns to keep' i stupaca dataframea dataframes[i]
    valid_columns = [col for col in columns_to_keep if col in dataframes[i].columns]
    dataframes[i] = dataframes[i][valid_columns]
    
print(dataframes[0].head())

Kombiniramo sve data frameove u jedan.

In [ ]:
df = pd.concat(dataframes, ignore_index=True)
print(df.head())

Možemo zapisati to u csv file.

In [ ]:
df.to_csv("data/election_results/kombinirani_rezultati.csv", index=False)

Sada ćemo grupirati rezultate po gradu/općini. Dosada su rezultati bili po biračkom mjestu. Također ćemo iz data framea izbaciti stupce s podatcima o biračkim mjestima.

In [ ]:
df = df.drop(columns=['Rbr BM', 'Naziv BM', 'Lokacija BM', 'Adresa BM'])
df = df.pivot_table(index=['Županija', 'Grad/općina/država'], aggfunc='sum').reset_index()

print(df.head())
df.to_csv("data/election_results/kombinirani_rezultati.csv", index=False)

Grad Zagreb je podijeljen u 3 izborne jednice, pa ćemo spojiti ta 3 retka u jedan.

In [ ]:
zagreb_rows = df[df['Županija'] == 'GRAD ZAGREB']
numerical_cols = zagreb_rows.select_dtypes(include='number').columns
zagreb_combined = zagreb_rows.groupby('Županija', as_index=False)[numerical_cols].sum()
zagreb_combined['Grad/općina/država'] = 'ZAGREB'

other_rows = df[df['Županija'] != 'GRAD ZAGREB']
df = pd.concat([zagreb_combined, other_rows], ignore_index=True)
df.to_csv("data/election_results/kombinirani_rezultati.csv", index=False)

Promijeniti ćemo redoslijed stupaca jer se poremetio tijekom ovih raznih operacija.

In [ ]:
columns = df.columns.to_list()
columns.insert(1, columns.pop(columns.index('Grad/općina/država')))
columns.insert(2, columns.pop(columns.index('Ukupno birača')))
columns.insert(3, columns.pop(columns.index('Glasovalo birača')))
columns.insert(4, columns.pop(columns.index('Glasovalo birača (po listićima)')))
columns.insert(5, columns.pop(columns.index('Važeći listići')))
columns.insert(5, columns.pop(columns.index('Nevažeći listići')))
df = df[columns]
df.to_csv("data/election_results/kombinirani_rezultati.csv", index=False)

Već smo vidjeli da se neke stranke pojavljuju više puta. Zato ćemo zbrojiti određene stupce. Također ćemo preimenovati neke stupce radi jednostavnosti - liste ćemo nazvati po najvećoj ili nekoliko najvećih stranaka od svih u listi. 

In [ ]:
for party in sorted(parties):
  print(party)

In [ ]:
df['AGRARNA STRANKA, AKCIJA ZA PROMJENE'] = df['AGRARNA STRANKA - POKRET ZA HRVATSKU POLJOPRIVREDU, AKCIJA ZA PROMJENE'] + df['AKCIJA ZA PROMJENE, AGRARNA STRANKA - POKRET ZA HRVATSKU POLJOPRIVREDU']

df = df.drop(columns=['AGRARNA STRANKA - POKRET ZA HRVATSKU POLJOPRIVREDU, AKCIJA ZA PROMJENE', 'AKCIJA ZA PROMJENE, AGRARNA STRANKA - POKRET ZA HRVATSKU POLJOPRIVREDU'])

df['DOMOVINSKI POKRET'] = df['DOMOVINSKI POKRET - DP, PRAVO I PRAVDA'] + df['DOMOVINSKI POKRET - DP, PRAVO I PRAVDA, BLOK ZA HRVATSKU - BLOK, AGRAMERI - NEZAVISNA LISTA'] + df['DOMOVINSKI POKRET - DP, PRAVO I PRAVDA, DEMOKRATSKI HSS - DHSS, ZELENA LISTA'] + df['DOMOVINSKI POKRET - DP, PRAVO I PRAVDA, DEMOKRATSKI HSS - DHSS, ZELENA LISTA, AGRAMERI - NEZAVISNA LISTA'] + df['DOMOVINSKI POKRET - DP, PRAVO I PRAVDA, NEZAVISNI, DEMOKRATSKI HSS - DHSS, ZELENA LISTA']

df = df.drop(columns=['DOMOVINSKI POKRET - DP, PRAVO I PRAVDA', 'DOMOVINSKI POKRET - DP, PRAVO I PRAVDA, BLOK ZA HRVATSKU - BLOK, AGRAMERI - NEZAVISNA LISTA', 'DOMOVINSKI POKRET - DP, PRAVO I PRAVDA, DEMOKRATSKI HSS - DHSS, ZELENA LISTA', 'DOMOVINSKI POKRET - DP, PRAVO I PRAVDA, DEMOKRATSKI HSS - DHSS, ZELENA LISTA, AGRAMERI - NEZAVISNA LISTA', 'DOMOVINSKI POKRET - DP, PRAVO I PRAVDA, NEZAVISNI, DEMOKRATSKI HSS - DHSS, ZELENA LISTA'])

df['SOCIJALDEMOKRATI, REFORMISTI, IDS, PGS'] = df['ISTARSKI DEMOKRATSKI SABOR - IDS, PRIMORSKO GORANSKI SAVEZ - PGS, UNIJA KVARNERA - UNIJA, ISTARSKA STRANKA UMIROVLJENIKA - PARTITO ISTRIANO DEI PENSIONATI - ISU - PIP, SOCIJALDEMOKRATI, NARODNA STRANKA - REFORMISTI - REFORMISTI, DEMOKRATI, HRVATSKI LABURISTI - STRANKA RADA - LABURISTI'] + df['NARODNA STRANKA - REFORMISTI - REFORMISTI, PRIMORSKO GORANSKI SAVEZ - PGS, ISTARSKI DEMOKRATSKI SABOR - IDS, UNIJA KVARNERA - UNIJA, STRANKA HRVATSKIH UMIROVLJENIKA - UMIROVLJENICI - UMIROVLJENICI'] + df['SOCIJALDEMOKRATI, HRVATSKI LABURISTI - STRANKA RADA - LABURISTI, DEMOKRATI'] + df['SOCIJALDEMOKRATI, ISTARSKI DEMOKRATSKI SABOR - IDS, PRIMORSKO GORANSKI SAVEZ - PGS, HRVATSKI LABURISTI - STRANKA RADA - LABURISTI, DEMOKRATI']

df = df.drop(columns=['ISTARSKI DEMOKRATSKI SABOR - IDS, PRIMORSKO GORANSKI SAVEZ - PGS, UNIJA KVARNERA - UNIJA, ISTARSKA STRANKA UMIROVLJENIKA - PARTITO ISTRIANO DEI PENSIONATI - ISU - PIP, SOCIJALDEMOKRATI, NARODNA STRANKA - REFORMISTI - REFORMISTI, DEMOKRATI, HRVATSKI LABURISTI - STRANKA RADA - LABURISTI', 'NARODNA STRANKA - REFORMISTI - REFORMISTI, PRIMORSKO GORANSKI SAVEZ - PGS, ISTARSKI DEMOKRATSKI SABOR - IDS, UNIJA KVARNERA - UNIJA, STRANKA HRVATSKIH UMIROVLJENIKA - UMIROVLJENICI - UMIROVLJENICI', 'SOCIJALDEMOKRATI, HRVATSKI LABURISTI - STRANKA RADA - LABURISTI, DEMOKRATI', 'SOCIJALDEMOKRATI, ISTARSKI DEMOKRATSKI SABOR - IDS, PRIMORSKO GORANSKI SAVEZ - PGS, HRVATSKI LABURISTI - STRANKA RADA - LABURISTI, DEMOKRATI'])

df['MOŽEMO'] = df['MOŽEMO! - POLITIČKA PLATFORMA'] + df['MOŽEMO! - POLITIČKA PLATFORMA, HOĆEMO PRAVEDNO'] + df['MOŽEMO! - POLITIČKA PLATFORMA, SRĐ JE GRAD']

df = df.drop(columns=['MOŽEMO! - POLITIČKA PLATFORMA', 'MOŽEMO! - POLITIČKA PLATFORMA, HOĆEMO PRAVEDNO', 'MOŽEMO! - POLITIČKA PLATFORMA, SRĐ JE GRAD'])

df['SDP'] = df['SOCIJALDEMOKRATSKA PARTIJA HRVATSKE - SDP, CENTAR, HRVATSKA SELJAČKA STRANKA - HSS, "DALIJA OREŠKOVIĆ I LJUDI S IMENOM I PREZIMENOM" - "DO i SIP", GRAĐANSKO-LIBERALNI SAVEZ - GLAS'] + df['SOCIJALDEMOKRATSKA PARTIJA HRVATSKE - SDP, CENTAR, HRVATSKA SELJAČKA STRANKA - HSS, "DALIJA OREŠKOVIĆ I LJUDI S IMENOM I PREZIMENOM" - "DO i SIP", NARODNA STRANKA - REFORMISTI - REFORMISTI, GRAĐANSKO-LIBERALNI SAVEZ - GLAS']

df = df.drop(columns=['SOCIJALDEMOKRATSKA PARTIJA HRVATSKE - SDP, CENTAR, HRVATSKA SELJAČKA STRANKA - HSS, "DALIJA OREŠKOVIĆ I LJUDI S IMENOM I PREZIMENOM" - "DO i SIP", GRAĐANSKO-LIBERALNI SAVEZ - GLAS', 'SOCIJALDEMOKRATSKA PARTIJA HRVATSKE - SDP, CENTAR, HRVATSKA SELJAČKA STRANKA - HSS, "DALIJA OREŠKOVIĆ I LJUDI S IMENOM I PREZIMENOM" - "DO i SIP", NARODNA STRANKA - REFORMISTI - REFORMISTI, GRAĐANSKO-LIBERALNI SAVEZ - GLAS'])

In [ ]:
# column_replacement_dict = {
#     'old_col1': 'new_col1',
#     'old_col2': 'new_col2',
#     'old_col3': 'new_col3'
# }
column_replacement_dict = {
    'AUTOHTONA - HRVATSKA STRANKA PRAVA - DRAŽEN KELEMINEC - A - HSP - Dražen Keleminec': 'A-HSP',
    'FOKUS, REPUBLIKA': 'FOKUS',
    'HRVATSKA DEMOKRATSKA ZAJEDNICA - HDZ, HRVATSKA SOCIJALNO - LIBERALNA STRANKA - HSLS, HRVATSKA DEMOKRŠĆANSKA STRANKA - HDS, HRVATSKA NARODNA STRANKA - LIBERALNI DEMOKRATI - HNS, HRVATSKA STRANKA UMIROVLJENIKA - HSU': 'HDZ',
    'HRVATSKA STRANKA PRAVA - HSP, HRVATSKO BILO, HRVATSKA DEMOKRATSKA SELJAČKA STRANKA - HDSS': 'HSP',
    'HRVATSKA STRANKA PRAVA DR. ANTE STARČEVIĆ': 'HSP AS',
    'MOST, HRVATSKI SUVERENISTI, HRVATSKA KONZERVATIVNA STRANKA - HKS, NEZAVISNA LISTA MLADIH - NLM': 'MOST',
    'UMIROVLJENICI ZAJEDNO - POLITIČKA PLATFORMA - UMIROVLJENICI ZAJEDNO, STRANKA UMIROVLJENIKA - SU, BLOK UMIROVLJENICI ZAJEDNO - BUZ, DEMOKRATSKA STRANKA UMIROVLJENIKA - DSU': 'UMIROVLJENICI ZAJEDNO'
}
df = df.rename(columns=column_replacement_dict)

In [ ]:
df.to_csv("data/election_results/kombinirani_rezultati.csv", index=False)

Sada ćemo izračunati postotke za svaku stranku u svakoj općini.

In [ ]:
parties = []

for i in range(7, 34):
      column_name = df.columns[i]
      parties.append(column_name)

new_columns = {}
for party in parties:
      new_columns[f'{party} %'] = round((df[party] / df['Važeći listići']) * 100, 2)

df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)

df.to_csv("data/election_results/kombinirani_rezultati.csv", index=False)